<a href="https://colab.research.google.com/github/rockey1006/Variational-Autoencoders/blob/master/Facenet_practice2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/erhwenkuo/face-recognition.git


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import tarfile
tar = tarfile.open('/content/drive/My Drive/lfw.tgz')
names = tar.getnames()
for name in names:
  tar.extract(name,path='/content/face-recognition/data/lfw')
tar.close()

In [0]:
%cd /content/face-recognition/


In [0]:
! git pull


In [0]:
! pip install facenet


In [0]:
%cd /content/face-recognition/

! pip install detect_face


In [0]:
from scipy import misc
import sys
import os
import random
from tqdm import tqdm

# 多維向量處理相關函式庫
import numpy as np

# 圖像處理相關函式庫
import cv2

# 深度學習相關函式庫
import tensorflow as tf

# 專案相關函式庫
import facenet
import detect_face

In [0]:
print(os.getcwd())

In [0]:
# 
ROOT_DIR = os.getcwd()#返回当前进程的工作目录:/content/face-recognition
DATA_PATH = os.path.join(ROOT_DIR, "data")#连接两个或更多的路径名:/content/face-recognition/data

# 模型的資料目錄
MODEL_PATH = os.path.join(ROOT_DIR, "model")

# MTCNN的模型
MTCNN_MODEL_PATH = os.path.join(MODEL_PATH, "mtcnn")

# 訓練/驗證用的圖像資料目錄
IMG_IN_PATH = os.path.join(DATA_PATH, "lfw","lfw")

# 訓練/驗證用的圖像資料目錄
IMG_OUT_PATH = os.path.join(DATA_PATH, "lfw_crops")

In [0]:
# 檢查要儲放裁剪結果的目錄
if not os.path.exists(IMG_OUT_PATH):
        os.makedirs(IMG_OUT_PATH)


# 臉類別(ImageClass)的列表與圖像路徑
dataset = facenet.get_dataset(IMG_IN_PATH)

In [0]:
print("Total face identities: ", len(dataset))


In [0]:
import detect_face


In [0]:
import warnings
warnings.filterwarnings('ignore')

# Utilities相關函式庫
import os
import sys
import random
from tqdm import tqdm
from scipy import misc

# 圖像處理/展現的相關函式庫
import cv2
import matplotlib.pyplot as plt

# 深度學習相關函式庫
import tensorflow as tf

In [0]:
# 專案的根目錄路徑
ROOT_DIR = os.getcwd()

# 訓練/驗證用的資料目錄
DATA_PATH = os.path.join(ROOT_DIR, "data")

# 訓練/驗證用的圖像資料目錄
TEST_IMGS_PATH = os.path.join(DATA_PATH, "images")

# 訓練/驗證用的影像資料目錄
TEST_VIDEOS_PATH = os.path.join(DATA_PATH, "videos")

In [0]:
minsize = 20 # 最小的臉部的大小
threshold = [ 0.6, 0.7, 0.7 ]  # 三個網絡(P-Net, R-Net, O-Net)的閥值
factor = 0.709 # scale factor

gpu_memory_fraction=1.0

In [0]:
!git clone https://github.com/ShyBigBoy/face-detection-mtcnn.git


In [46]:
print(os.getcwd())

/content/face-recognition/face-detection-mtcnn


In [49]:
!pip install detect_face

ERROR: Could not find a version that satisfies the requirement detect_face (from versions: none)
ERROR: No matching distribution found for detect_face


In [54]:
import shutil, sys  

shutil.move("/content/face-recognition/face-detection-mtcnn/det1.npy", "/content/face-recognition/det1.npy") 


'/content/face-recognition/det1.npy'

In [0]:
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [59]:
shutil.move("/content/det1.npy", "/content/face-recognition/det1.npy") 

'/content/face-recognition/det1.npy'

In [60]:
shutil.move("/content/det2.npy", "/content/face-recognition/det2.npy") 

'/content/face-recognition/det2.npy'

In [61]:
shutil.move("/content/det3.npy", "/content/face-recognition/det3.npy") 

'/content/face-recognition/det3.npy'

In [62]:
print('Creating networks and loading parameters')
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
    with sess.as_default():
        pnet, rnet, onet = detect_face.create_mtcnn(sess, None)

Creating networks and loading parameters


In [0]:
minsize = 20  # 最小的臉部的大小
threshold = [0.6, 0.7, 0.7]  # 三個網絡(P-Net, R-Net, O-Net)的閥值
factor = 0.709  # scale factor

margin = 44 # 在裁剪人臉時的邊框margin
image_size = 182 # 160 + 22

In [0]:
# 將一個隨機key添加到圖像檔名以允許使用多個進程進行人臉對齊
random_key = np.random.randint(0, high=99999)
bounding_boxes_filename = os.path.join(IMG_OUT_PATH, 'bounding_boxes_%05d.txt' % random_key)

In [0]:
from matplotlib.pyplot import imread
from skimage.transform import resize
import imageio

# 使用Tensorflow來運行MTCNN
with open(bounding_boxes_filename, "w") as text_file:
    nrof_images_total = 0 # 處理過的圖像總數
    nrof_successfully_aligned = 0 # 人臉圖像align的總數
    
    # 迭代每一個人臉身份(ImageClass)
    for cls in tqdm(dataset):
        output_class_dir = os.path.join(IMG_OUT_PATH, cls.name) # 裁剪後的圖像目錄
        if not os.path.exists(output_class_dir):
            os.makedirs(output_class_dir)
        
        # 迭代每一個人臉身份的圖像的路徑 (ImageClass.image_paths)
        for image_path in cls.image_paths:
            nrof_images_total += 1
            filename = os.path.splitext(os.path.split(image_path)[1])[0] # 取得圖像檔名
            output_filename = os.path.join(output_class_dir, filename + '.png') # 設定輸出的圖像檔名            
            # print(image_path)
            
            if not os.path.exists(output_filename):
                try:
                    img = imread(image_path) # 讀進圖檔
                    # print('read data dimension: ', img.ndim)                    
                except (IOError, ValueError, IndexError) as e:
                    errorMessage = '{}: {}'.format(image_path, e)
                    # print(errorMessage)
                else:
                    # 將圖檔轉換成numpy array (height, widith, color_channels)
                    if img.ndim < 2:
                        print('Unable to align "%s"' % image_path)
                        text_file.write('%s\n' % (output_filename))
                        continue
                    if img.ndim == 2:
                        img = facenet.to_rgb(img)
                        print('to_rgb data dimension: ', img.ndim)
                    img = img[:, :, 0:3]
                    # print('after data dimension: ', img.ndim)
                    
                    # 使用MTCNN來偵測人臉在圖像中的位置
                    bounding_boxes, _ = detect_face.detect_face(img, minsize, pnet, rnet, onet, threshold, factor)
                    nrof_faces = bounding_boxes.shape[0] # 偵測到的人臉總數
                    # print('detected_face: %d' % nrof_faces)
                    if nrof_faces > 0:
                        # 當有偵測到多個人臉的時候, 我們希望從中找到主要置中位置的人臉
                        det = bounding_boxes[:, 0:4]
                        img_size = np.asarray(img.shape)[0:2]
                        if nrof_faces > 1:
                            bounding_box_size = (det[:, 2] - det[:, 0]) * (det[:, 3] - det[:, 1])
                            img_center = img_size / 2
                            offsets = np.vstack([(det[:, 0] + det[:, 2]) / 2 - img_center[1],
                                                 (det[:, 1] + det[:, 3]) / 2 - img_center[0]])
                            offset_dist_squared = np.sum(np.power(offsets, 2.0), 0)
                            index = np.argmax(bounding_box_size - offset_dist_squared * 2.0)  # some extra weight on the centering
                            det = det[index, :]
                        det = np.squeeze(det)
                        bb_temp = np.zeros(4, dtype=np.int32)
                        # 取得人臉的左上角與右下角座標
                        bb_temp[0] = det[0]
                        bb_temp[1] = det[1]
                        bb_temp[2] = det[2]
                        bb_temp[3] = det[3]
                        
                        # 進行裁剪以及大小的轉換
                        cropped_temp = img[bb_temp[1]:bb_temp[3], bb_temp[0]:bb_temp[2], :]
                        scaled_temp = resize(cropped_temp, (image_size, image_size))

                        nrof_successfully_aligned += 1
                        imageio.imwrite(output_filename, scaled_temp) # 儲存處理過的圖像
                        text_file.write('%s %d %d %d %d\n' % (output_filename, bb_temp[0], bb_temp[1], bb_temp[2], bb_temp[3]))
                    else:
                        # print('Unable to align "%s"' % image_path)
                        text_file.write('%s\n' % (output_filename))

print('Total number of images: %d' % nrof_images_total)
print('Number of successfully aligned images: %d' % nrof_successfully_aligned)

 78%|███████▊  | 4456/5749 [31:17<08:10,  2.63it/s]WARNING:root:Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [0]:
import matplotlib.pyplot as plt

face_identity = 'Bill_Clinton'
origin_face_image = os.path.join(IMG_IN_PATH, face_identity, 'Bill_Clinton_0009.jpg')
aligned_face_image = os.path.join(IMG_OUT_PATH, face_identity, 'Bill_Clinton_0009.png')